# Bias and limitations

Amacınız üretimde ön eğitimli bir model veya ince ayarlı bir versiyon kullanmaksa, lütfen bu modellerin güçlü araçlar olmalarına rağmen sınırlamaları olduğunu unutmayın. Bunların en büyüğü, büyük miktarda veri üzerinde ön eğitime olanak sağlamak için araştırmacıların genellikle bulabildikleri tüm içeriği kazıyarak internette mevcut olanın en iyisini ve en kötüsünü almasıdır.

Hızlı bir örnek vermek gerekirse, BERT modeli ile bir fill-mask pipeline örneğine geri dönelim:

In [1]:
from transformers import pipeline

unmasker = pipeline("fill-mask", model="bert-base-uncased")
result = unmasker("This man works as a [MASK].")
result

2024-08-01 18:43:55.698922: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 18:43:55.699079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 18:43:55.868216: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'score': 0.07510633766651154,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'this man works as a carpenter.'},
 {'score': 0.04641929641366005,
  'token': 5160,
  'token_str': 'lawyer',
  'sequence': 'this man works as a lawyer.'},
 {'score': 0.03914567083120346,
  'token': 7500,
  'token_str': 'farmer',
  'sequence': 'this man works as a farmer.'},
 {'score': 0.03280145302414894,
  'token': 6883,
  'token_str': 'businessman',
  'sequence': 'this man works as a businessman.'},
 {'score': 0.02929236739873886,
  'token': 3460,
  'token_str': 'doctor',
  'sequence': 'this man works as a doctor.'}]

In [2]:
print([r["token_str"] for r in result])

['carpenter', 'lawyer', 'farmer', 'businessman', 'doctor']


In [3]:
result = unmasker("This woman works as a [MASK].")
print([r["token_str"] for r in result])

['nurse', 'maid', 'teacher', 'waitress', 'prostitute']


Bu iki cümledeki eksik kelimeyi doldurması istendiğinde, model sadece bir tane cinsiyetsiz cevap veriyor (garson/garson). Diğerleri genellikle belirli bir cinsiyetle ilişkilendirilen iş meslekleridir - ve evet, fahişe modelin "kadın" ve "iş" ile ilişkilendirdiği ilk 5 olasılık arasına girmiştir. BERT, internetin her yerinden veri kazıyarak değil, görünüşte tarafsız veriler kullanarak (İngilizce Wikipedia ve BookCorpus veri kümeleri üzerinde eğitilmiştir) oluşturulmuş nadir Transformer modellerinden biri olmasına rağmen bu durum gerçekleşmektedir.

**Bu nedenle, bu araçları kullanırken, kullandığınız orijinal modelin çok kolay bir şekilde cinsiyetçi, ırkçı veya homofobik içerik üretebileceğini aklınızın bir köşesinde tutmanız gerekir**. Verileriniz üzerinde modele ince ayar yapmak bu içsel önyargıyı ortadan kaldırmayacaktır.